In [16]:
from shapely.geometry import Polygon
import numpy as np
import cv2
from PIL import Image
import math
import os
import torch
import torchvision.transforms as transforms
from torch.utils import data

from dataset import extract_vertices, adjust_height, rotate_img, crop_img, get_score_geo

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device

device(type='cuda', index=0)

In [18]:
class custom_dataset(data.Dataset):
    def __init__(self, img_path, gt_path, scale=0.25, length=512):
        super(custom_dataset, self).__init__()
        self.img_files = [os.path.join(img_path, img_file) for img_file in sorted(os.listdir(img_path))]
        self.gt_files  = [os.path.join(gt_path, gt_file) for gt_file in sorted(os.listdir(gt_path))]
        self.scale = scale
        self.length = length
  
    def __len__(self):
	    return len(self.img_files)

    def __getitem__(self, index):
        with open(self.gt_files[index], 'r', encoding='UTF-8') as f:
            lines = f.readlines()
        vertices, labels = extract_vertices(lines)
		
        img = Image.open(self.img_files[index])
        img, vertices = adjust_height(img, vertices) 
        img, vertices = rotate_img(img, vertices)
        img, vertices = crop_img(img, vertices, labels, self.length) 
        transform = transforms.Compose([transforms.ColorJitter(0.5, 0.5, 0.5, 0.25), \
                                        transforms.ToTensor(), \
                                        transforms.Normalize(mean=(0.5,0.5,0.5),std=(0.5,0.5,0.5))])
        
        score_map, geo_map, ignored_map = get_score_geo(img, vertices, labels, self.scale, self.length)
        return transform(img), score_map, geo_map, ignored_map

In [19]:
train_img_path = os.path.abspath('D:/_data/personal_project/EAST_test_data/train_img')
train_gt_path  = os.path.abspath('D:/_data/personal_project/EAST_test_data/train_gt')

In [43]:
img_files = [os.path.join(train_img_path, img_file) for img_file in sorted(os.listdir(train_img_path))]

len(img_files)

32

In [47]:
trainset = custom_dataset(train_img_path, train_gt_path)   # x, y값이 들어있는 튜플
train_loader = data.DataLoader(trainset, batch_size=1, shuffle=True, num_workers=0, drop_last=True)

In [73]:
a = next(iter(train_loader))

In [61]:
import matplotlib.pyplot as plt

# 이미지와 정답(label)을 표시합니다.
train_features, train_labels = next(iter(train_loader))
a = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

ValueError: too many values to unpack (expected 2)